# Preparação

## Lembretes e imports

In [ ]:
# Incluir a parte do sumário porque tô vendo que vai dar merda se eu não fizer isso logo
# Checar com outras mãos pra encontrar erros e inconsistências
# quando a pessoa folda e mostra, eu não tô pegando a(s) carta(s) que ela mostrou. por pura pregs mesmo

In [ ]:
## Imports

import re
import json
import pprint
import os
import pandas as pd

## Variáveis Universais

In [ ]:
SOURCE_FOLDER = r"/content/drive/MyDrive/Poker/hands_backup/"
JSON_FOLDER = r"/content/drive/MyDrive/Projetos/poker_app/json_files/"
JSON_FOLDER_V2 = r"/content/drive/MyDrive/Projetos/poker_app/json_files_v2/" # criei esse pra subir os arquivos no loop completo

HOJE = (pd.Timestamp.today() - pd.Timedelta(hours=3)).strftime("%Y-%m-%d")
AGORA =(pd.Timestamp.today() - pd.Timedelta(hours=3)).strftime("%Y-%m-%d %r")

## Puxar arquivo .txt

In [ ]:
all_raw_files_list = os.listdir(SOURCE_FOLDER)

ms_sng_files = [] # microstakes sit n go

for file in all_raw_files_list:
  if "$0.44" in file or "$0.88" in file:
    ms_sng_files.append(file)

#pprint.pprint(ms_sng_files, sort_dicts=False)

In [ ]:
filename_checker = pd.DataFrame({'filename': pd.Series(ms_sng_files)})

filename_checker.query("filename.str.contains('3684678181')")

,filename
87,HH20231210 T3684678181 No Limit Hold'em $0.88 ...


In [ ]:
current_file = ms_sng_files[89]

print(current_file)

HH20231216 T3687289002 No Limit Hold'em $0.88 + $0.12.txt


# Extração básica

### Separar .txt em blocos

In [ ]:
# Separa as mãos em estágios (setup, pre-flop etc)
# retorna uma lista de dicionários. key = stage, value = raw lines

def break_hands_stages_v2(file):
  complete_filepath = f"{SOURCE_FOLDER}{file}"
  raw_hands_data = []
  table_hand_counter = 0
  hand_list = []
  raw_hands_stages = []

  # abre o arquivo e faz o tratamento básico de tirar espaços em branco etc
  with open(complete_filepath) as raw_text_game_data:
    table_strings = raw_text_game_data.readlines()
    clean_table_strings = [line.strip() for line in table_strings]
    no_empty_lines = list(filter(None, clean_table_strings))

    # quebra o blocão de texto em mãos
    # gera uma lista de listas em que cada sub-lista é uma linha de texto
    for line in no_empty_lines:
      if "PokerStars" in line:
        if table_hand_counter != 0:
          raw_hands_data.append(hand_list)
          hand_list = []
        table_hand_counter += 1
        hand_list.clear()
      hand_list.append(line)

  raw_hands_data.append(hand_list)

  # quebra o bloco de texto da mão completa em cada estágio do jogo
  for hand in raw_hands_data:
    raw_dict = {
      'Metadata': [],
      'Setup': [],
      'Pre_flop': [],
      'Flop': [],
      'Turn': [],
      'River': [],
      'Showdown': [],
      'Summary': []
    }
    for line in hand:
      if "PokerStars" in line:
        stage = 'Setup'
      if "*** HOLE CARDS ***" in line:
        stage = "Pre_flop"
      if "*** FLOP ***" in line:
        stage = "Flop"
      if "TURN" in line:
        stage = "Turn"
      if "RIVER" in line:
        stage = "River"
      if "SHOW DOWN" in line:
        stage = "Showdown"
      if "SUMMARY" in line:
        stage = "Summary"
      raw_dict[stage].append(line)
    raw_hands_stages.append(raw_dict)

  # tira as duas primeiras linhas do Setup e joga pro Metadata. Achei mais fácil
  for hand in raw_hands_stages:
    hand['Metadata'].append(hand['Setup'][0])
    hand['Metadata'].append(hand['Setup'][1])
    hand['Setup'].pop(0)
    hand['Setup'].pop(0)

  return raw_hands_stages

In [ ]:
#pprint.pprint(break_hands_stages_v2(current_file), sort_dicts=False)

### Metadados partida

In [ ]:
# extrai metadados do torneio

def get_match_metadata(list_split_hands):
  raw_hands_stages = list_split_hands
  header = raw_hands_stages[0]['Metadata'][0]
  split_header = header.split()

  match_metadata = {'id': re.search(r"Tournament #(\d+)", header).group(1),
                  'platform': 'PokerStars',
                  'match_type': 'tournament',
                  'buy_in': 0.50,
                  'currency': 'USD',
                  'poker_type': 'Texas Holdem',
                  'bet_limit': "No",
                  'start_datetime': f"{split_header[-3]} {split_header[-2]}",
                  'timezone': 'ET'
                    }

  return match_metadata

#pprint.pprint(match_dict)

### Metadados mão

In [ ]:
# declara a função que extrai os metadados da mão

def get_hand_metadata(hand):
  header1 = hand['Metadata'][0]
  header2 = hand['Metadata'][1]
  split_header1 = header1.split()

  hand_metadata = {
    'id': re.search(r'\d{12}', header1).group(),
    'match_id': re.search(r"Tournament #(\d+)", header1).group(1),
    'table_id': re.search(r'\d+\s\d+', header2).group(),
    'current_lvl': re.search(r'Level\s+(\w+)', header1).group(1),
    'price_sb': re.search(r'\(\d+/\d+\)', header1).group().strip('()').split("/")[0],
    'price_bb': re.search(r'\(\d+/\d+\)', header1).group().strip('()').split("/")[1],
    'start_datetime': f"{split_header1[-3]} {split_header1[-2]}",
    'max_players': re.search(r'\d+-max', header2).group().split("-")[0],
    'button_seat': re.search(r'Seat #(\d+)', header2).group(1)
  }

  return hand_metadata

#get_hand_metadata(raw_hands_stages[0])

### Setup

In [ ]:
# Refazendo o método que extrai o setup pq resolvi mudar a estrutura

def new_get_setup_data(raw_setup_data):
  new_setup_data = {
      'seats': [],
      'forced_bets': []
  }

  for line in raw_setup_data:
    if "Seat" in line:
      seat_dict = {
          'number': re.search(r'Seat (\d+):', line).group(1),
          'player': re.search(r':\s(.+?)\s\(', line).group(1),
          'chips': re.search(r'\((\d+)\s', line).group(1),
          'starts_sitting_out': ""
      }
      if "sitting" in line:
        seat_dict['starts_sitting_out'] = "Yes"
      new_setup_data['seats'].append(seat_dict)
    else:
      forced_bet_dict = {
          'player': re.search(r'^(.*):', line).group(1),
          'action_description': "",
          'amount': re.search(r'\b\d+\b', line).group(),
          'all_in': ""
      }
      if "small blind" in line:
        forced_bet_dict['action_description'] = "posts small_blind"
      elif "big blind" in line:
        forced_bet_dict['action_description'] = "posts big_blind"
      elif "ante" in line:
        forced_bet_dict['action_description'] = "posts ante"
      if "all-in" in line:
        forced_bet_dict['all_in'] = "Y"
      new_setup_data['forced_bets'].append(forced_bet_dict)


  return new_setup_data

#pprint.pprint(new_get_setup_data(raw_hands_stages[54]['Setup']), sort_dicts=False) # teste
#print(raw_hands_stages[54]['Setup'])

### Ações

In [ ]:
# extrai os dados das ações dos jogadores

def get_actions_data(list):
  action_counter = 0
  actions_list = []

  for line in list:
    action_counter += 1

    action_dict = {
        'action_number': action_counter,
        'player': "",
        'action_description': "",
        'amount': "",
        'new_amount': "",
        'all_in': ""
    }

    if "checks" in line:
      action_dict['player'] = re.search(r'^(.*):', line).group(1)
      action_dict['action_description'] = 'checks'
    elif "folds" in line:
      action_dict['player'] = re.search(r'^(.*):', line).group(1)
      action_dict['action_description'] = 'folds'
    elif "calls" in line:
      action_dict['player'] = re.search(r'^(.*):', line).group(1)
      action_dict['action_description'] = 'calls'
      action_dict['amount'] = re.search(r'(?<=calls\s)\d+', line).group()
    elif "bets" in line:
      action_dict['player'] = re.search(r'^(.*):', line).group(1)
      action_dict['action_description'] = 'bets'
      action_dict['amount'] = re.search(r'(?<=bets\s)\d+', line).group()
    elif "raises" in line:
      action_dict['player'] = re.search(r'^(.*):', line).group(1)
      action_dict['action_description'] = 'raises'
      action_dict['amount'] = re.search(r'\b\d+\b(?=\s+to)', line).group()
      action_dict['new_amount'] = re.search(r'(?<=to\s)\d+', line).group()
    elif "Uncalled" in line:
      action_dict['player'] = re.search(r'(?<=to\s)(.*)', line).group(1)
      action_dict['action_description'] = "retakes"
      action_dict['amount'] = re.search(r'\((\d+)\)', line).group(1)
      action_dict['all_in'] = ""
    elif "collected" in line:
      action_dict['player'] = re.search(r'(.*)\scollected', line).group(1)
      action_dict['action_description'] = "collects"
      action_dict['amount'] = re.search(r'(?<=collected\s)\d+', line).group()
      action_dict['all_in'] = ""
    elif "is disconnected" in line:
      action_dict['player'] = re.search(r'(.*)\sis', line).group(1)
      action_dict['action_description'] = "disconnects"
    elif "is connected" in line:
      action_dict['player'] = re.search(r'(.*)\sis', line).group(1)
      action_dict['action_description'] = "connects"
    elif "timed out" in line:
      action_dict['player'] = re.search(r'(.*)\shas', line).group(1)
      action_dict['action_description'] = "times_out"
    elif "sitting" in line:
      action_dict['player'] = re.search(r'(.*)\sis', line).group(1)
      action_dict['action_description'] = "sits_out"
    elif "returned" in line:
      action_dict['player'] = re.search(r'(.*)\shas', line).group(1)
      action_dict['action_description'] = "returns_to_table"
    elif "doesn't show" in line:
      action_dict['player'] = re.search(r'^(.*):', line).group(1)
      action_dict['action_description'] = "doesnt_show_hand"
    elif "finished" in line:
      placement = re.search(r'tournament in (\w+)', line).group(1)
      action_dict['player'] = re.search(r'(.*)\bfinished', line).group(1).strip()
      action_dict['action_description'] = f"finishes {placement}"
    elif "shows" in line:
      action_dict['player'] = re.search(r'^(.*):', line).group(1)
      sd_cards = re.search(r'\[(.*?)\]', line).group(1)
      action_dict['action_description'] = f"shows {sd_cards}"

    if "all-in" in line:
      action_dict['all_in'] = "Y"

    actions_list.append(action_dict)

  return actions_list

#get_actions_data(break_hands_stages()[1]["Showdown"]) # teste

In [ ]:
#le_dict = break_hands_stages()
#le_dict

#for line in le_dict:
#  print(line['Metadata'])

#le_dict[-2]
#line = 'UEHJU3 collected 990 from pot'
#amount = re.search(r'(?<=collected\s)\d+', line).group()
#amount

### Streets

In [ ]:
# método que extrai os dados de cada street
# meio feio mas fiquei com pregs de ajeitar o regex, então tá valendo

def get_street_data(street_list_raw):
  street_data = {
      'actions': []
  }
  if len(street_list_raw) == 0:
    return street_data
  else:
    street_header = street_list_raw[0]
    street_data = {}
    if "HOLE" in street_header:
      street_data['hero_cards'] = re.search(r'\[(.*?)\]', street_list_raw[1]).group(1).split()
      street_data['actions'] = get_actions_data(street_list_raw[2:])
    elif "FLOP" in street_header:
      street_data['street_cards'] = re.search(r'\[(.*)\]', street_list_raw[0]).group(1).split()
      street_data['actions'] = get_actions_data(street_list_raw[1:])
    elif "TURN" in street_header:
      street_data['street_cards'] = re.search(r'\[(\w+)\]', street_list_raw[0]).group(1)
      street_data['actions'] = get_actions_data(street_list_raw[1:])
    elif "RIVER" in street_header:
      street_data['street_cards'] = re.search(r'\[(\w+)\]', street_list_raw[0]).group(1)
      street_data['actions'] = get_actions_data(street_list_raw[1:])
    else:
      return street_data
    return street_data

#pprint.pprint(get_street_data(raw_hands_stages[0]['Flop']))

### Showdown

In [ ]:
def new_get_showdown_data(showdown_list_raw):
  sd_actions_list = []

  if len(showdown_list_raw) > 0:
    sd_action_counter = 0
    for line in showdown_list_raw[1:]:
      sd_action_counter += 1
      sd_action_dict = {
            'action_number': sd_action_counter,
            'player': "",
            'action_description': "",
            'amount': "",
            'new_amount': "",
            'all_in': "",
            'sd_cards': [],
            'sd_finalhand': ""
      }
      if "shows" in line:
        sd_action_dict['player'] = re.search(r'^(.*):', line).group(1)
        sd_action_dict['action_description'] = "shows_hand"
        sd_action_dict['sd_cards'] = re.search(r'\[(.*?)\]', line).group(1).split()
        sd_action_dict['sd_finalhand'] = re.search(r'\((.*?)\)', line).group(1)
      elif "mucks" in line:
        sd_action_dict['player'] = re.search(r'^(.*):', line).group(1)
        sd_action_dict['action_description'] = "mucks_hand"
      elif "collected" in line:
        if "side" in line:
          pot_type = "side"
        else:
          pot_type = "main"
        sd_action_dict['player'] = re.search(r'(.*)\bcollected', line).group(1).strip()
        sd_action_dict['action_description'] = f"collects_pot {pot_type}"
        sd_action_dict['amount'] = re.search(r'collected (\d+)', line).group(1)
      elif "finished" in line:
        sd_action_dict['player'] = re.search(r'(.*)\bfinished', line).group(1).strip()
        placement = re.search(r'tournament in (\w+)', line).group(1)
        sd_action_dict['action_description'] = f"finishes {placement}"
      elif "wins" in line:
        sd_action_dict['player'] = re.search(r'(.*)\bwins', line).group(1).strip()
        sd_action_dict['action_description'] = "finishes 1st"
      elif "is disconnected" in line:
        sd_action_dict['player'] = re.search(r'(.*)\sis', line).group(1)
        sd_action_dict['action_description'] = "disconnects"
      elif "returned" in line:
        sd_action_dict['player'] = re.search(r'(.*)\shas', line).group(1)
        sd_action_dict['action_description'] = "returns_to_table"
      sd_actions_list.append(sd_action_dict)

  return sd_actions_list

#pprint.pprint(raw_hands_stages[3]['Showdown'])
#pprint.pprint(new_get_showdown_data(raw_hands_stages[2]['Showdown']), sort_dicts=False)

### Summary

In [ ]:
def get_summary_data(summary_list):
  summary_data = {
      'summary_general': {},
      'summary_seats': []
  }

  raw_summary_general_data = []
  raw_summary_seats_data = []
  for line in summary_list:
    if "Seat " in line:
      raw_summary_seats_data.append(line)
    else:
      raw_summary_general_data.append(line)

  for line in raw_summary_general_data:
    if "Total" in line:
      summary_data['summary_general']['total_pot'] = re.search(r'pot (\d+)', line).group(1)
      summary_data['summary_general']['rake'] = re.search(r'Rake (\d+)', line).group(1)
    if "Board" in line:
      summary_data['summary_general']['board'] = re.search(r'\[(.*?)\]', line).group(1).split()

  for line in raw_summary_seats_data:
    summary_seat_dict = {
          'number': re.search(r'Seat (\d+):', line).group(1),
          'action_description': "",
          'action_phrase': ""
    }
    if "folded" in line:
      summary_seat_dict['action_description'] = "folded"
      summary_seat_dict['action_phrase'] = re.search(r'(?<=folded ).*$', line).group() #re.search(r'folded (.*)', line).group(1)
    elif "mucked" in line:
      summary_seat_dict['action_description'] = "mucked"
      summary_seat_dict['action_phrase'] = re.search(r'(?<=mucked ).*$', line).group()
    elif "showed" in line:
      summary_seat_dict['action_description'] = "showed"
      summary_seat_dict['action_phrase'] = re.search(r'(?<=showed ).*$', line).group()
    summary_data['summary_seats'].append(summary_seat_dict)

  return summary_data


#pprint.pprint(raw_hands_stages[0]['Summary'])
#pprint.pprint(get_summary_data(raw_hands_stages[1]['Summary']), sort_dicts=False)

### Loop extrações

In [ ]:
# LOOP PRINCIPAL
# esse é o loop que vai passar mão por mão executando os métodos de extração

def get_all_hands_data(file):
  raw_hands_stages = break_hands_stages_v2(file)
  match_dict = {
    'match_metadata': get_match_metadata(raw_hands_stages),
    'hands': {}
  }
  for hand in raw_hands_stages:
    temp_hand_metadata = get_hand_metadata(hand)
    hand_id = f"PS_hand_{temp_hand_metadata['id']}"
    match_dict['hands'][hand_id] = {}
    match_dict['hands'][hand_id]['Metadata'] = temp_hand_metadata
    match_dict['hands'][hand_id]['Setup'] = new_get_setup_data(hand['Setup'])
    match_dict['hands'][hand_id]['Pre_flop'] = get_street_data(hand['Pre_flop'])
    match_dict['hands'][hand_id]['Flop'] = get_street_data(hand['Flop'])
    match_dict['hands'][hand_id]['Turn'] = get_street_data(hand['Turn'])
    match_dict['hands'][hand_id]['River'] = get_street_data(hand['River'])
    match_dict['hands'][hand_id]['Showdown'] = new_get_showdown_data(hand['Showdown'])
    match_dict['hands'][hand_id]['Summary'] = get_summary_data(hand['Summary'])

  return match_dict

In [ ]:
#pprint.pprint(get_all_hands_data(current_file), sort_dicts=False)

# Transformações pro novo dicionário

### Metadata

In [ ]:
def transform_hand_metadata(hand):
  t_hand_metadata = hand['Metadata'] # t de "transformed"
  t_hand_metadata['hero_card1'] = hand['Pre_flop']['hero_cards'][0]
  t_hand_metadata['hero_card2'] = hand['Pre_flop']['hero_cards'][1]
  t_hand_metadata['flop_card1'] = ""
  t_hand_metadata['flop_card2'] = ""
  t_hand_metadata['flop_card3'] = ""
  t_hand_metadata['turn_card'] = ""
  t_hand_metadata['river_card'] = ""

  if len(hand['Flop']['actions']) > 0:
    t_hand_metadata['flop_card1'] = hand['Flop']['street_cards'][0]
    t_hand_metadata['flop_card2'] = hand['Flop']['street_cards'][1]
    t_hand_metadata['flop_card3'] = hand['Flop']['street_cards'][2]
  if len(hand['Turn']['actions']) > 0:
    t_hand_metadata['turn_card'] = hand['Turn']['street_cards']
  if len(hand['River']['actions']) > 0:
    t_hand_metadata['river_card'] = hand['River']['street_cards']

  return t_hand_metadata

#pprint.pprint(transform_hand_metadata(match_dict['hands']['PS_hand_239454817006']), sort_dicts = False)
#pprint.pprint(match_dict['hands']['PS_hand_239454817006'], sort_dicts = False)

### Seats

In [ ]:
def transform_seats_data(hand):
  t_seats = []
  for seat in hand['Setup']['seats']:
    t_seat_data = {
        'match_id': hand['Metadata']['match_id'],
        'hand_id': hand['Metadata']['id'],
        'seat_number': seat['number'],
        'seat_player': seat['player'],
        'seat_chips': seat['chips'],
        'starts_sitting_out': seat['starts_sitting_out']
    }
    t_seats.append(t_seat_data)
  return t_seats

#pprint.pprint(transform_seats_data(match_dict['hands']['PS_hand_239454817006']), sort_dicts = False)
#print(match_dict['hands'])
#pprint.pprint(match_dict['hands']['PS_hand_239454817006'], sort_dicts = False)

### Forced Bets

In [ ]:
def transform_forced_bets_data(hand):
  t_forced_bets = []

  action_counter = 0
  for bet in hand['Setup']['forced_bets']:
    action_counter += 1
    t_forced_bet_data = {
        'match_id': hand['Metadata']['match_id'],
        'hand_id': hand['Metadata']['id'],
        'stage': 'forced_bets',
        'action_number': action_counter,
        'player': bet['player'],
        'action_description': bet['action_description'],
        'amount': bet['amount'],
        'new_amount': '',
        'all_in': bet['all_in']
    }
    t_forced_bets.append(t_forced_bet_data)
  return t_forced_bets

#pprint.pprint(transform_forced_bets_data(match_dict['hands']['PS_hand_239592712781']), sort_dicts = False)

### Actions

In [ ]:
def transform_actions_data(hand, stage):
  t_actions = []

  for action in hand[stage]['actions']:
    t_action_data = {
        'match_id': hand['Metadata']['match_id'],
        'hand_id': hand['Metadata']['id'],
        'stage': stage.lower(),
        'action_number': action['action_number'],
        'player': action['player'],
        'action_description': action['action_description'],
        'amount': action['amount'],
        'new_amount': action['new_amount'],
        'all_in': action['all_in']
    }
    t_actions.append(t_action_data)
  return t_actions

#pprint.pprint(transform_actions_data(match_dict['hands']['PS_hand_239776079265'], 'Pre_flop'), sort_dicts = False)

### Showdown

In [ ]:
def transform_showdown_actions(hand):
  t_showdown_actions = []
  for action in hand['Showdown']:
    t_sd_action_data = {
        'match_id': hand['Metadata']['match_id'],
        'hand_id': hand['Metadata']['id'],
        'stage': "showdown",
        'action_number': action['action_number'],
        'player': action['player'],
        'action_description': action['action_description'],
        'amount': action['amount'],
        'new_amount': "",
        'all_in': ""
    }
    t_showdown_actions.append(t_sd_action_data)
  return t_showdown_actions

def transform_showdown_cards(hand):
  t_showdown_cards = []
  for showed_cards in hand['Showdown']['cards_shown']:
    t_sd_cards_data = {
        'match_id': hand['Metadata']['match_id'],
        'hand_id': hand['Metadata']['id'],
        'player': showed_cards['player'],
        'card1': showed_cards['card1'],
        'card2': showed_cards['card2'],
        'hand_result': showed_cards['hand_result']
    }
    t_showdown_cards.append(t_sd_cards_data)
  return t_showdown_cards

#pprint.pprint(transform_showdown_actions(match_dict['hands']['PS_hand_239454500194']), sort_dicts = False)
#pprint.pprint(match_dict['hands']['PS_hand_239454500194']['Showdown'], sort_dicts = False)

### Summary

In [ ]:
def transform_summary_seat_data(hand):
  t_summary_seat_data = []
  return t_summary_seat_data

#pprint.pprint(transform_summary_seat_data(game_dict['hands']['PS_hand_239454500194']), sort_dicts = False)
#pprint.pprint(game_dict['hands']['PS_hand_239454500194']['Summary'], sort_dicts = False)

### Loop transformações

In [ ]:
def transform_all_data(file):
  extracted_data = get_all_hands_data(file)

  final_game_dict = {
      'match_metadata': extracted_data['match_metadata'],
      'hands': []
  }

  for current_hand in extracted_data['hands']:
    whole_hand_name = extracted_data['hands'][current_hand]
    #print(current_hand) ##
    new_hand_dict = {
        'metadata': transform_hand_metadata(whole_hand_name),
        'seats': transform_seats_data(whole_hand_name),
        'forced_bets': transform_forced_bets_data(whole_hand_name),
        'preflop_actions': transform_actions_data(whole_hand_name, 'Pre_flop'),
        'flop_actions': transform_actions_data(whole_hand_name, 'Flop'),
        'turn_actions': transform_actions_data(whole_hand_name, 'Turn'),
        'river_actions': transform_actions_data(whole_hand_name, 'River'),
        'showdown_actions': transform_showdown_actions(whole_hand_name),
        'summary': whole_hand_name['Summary']['summary_seats']
        #'shown_cards': transform_showdown_cards(whole_hand_name)
    }

    #print(new_hand_dict) ##

    new_hand_dict['metadata']['total_pot'] = whole_hand_name['Summary']['summary_general']['total_pot']
    new_hand_dict['metadata']['rake'] = whole_hand_name['Summary']['summary_general']['rake']

    final_game_dict['hands'].append(new_hand_dict)

  return final_game_dict


#pprint.pprint(match_dict['hands']['PS_hand_239454567539']['Showdown'], sort_dicts = False)

In [ ]:
#pprint.pprint(transform_all_data(current_file), sort_dicts = False)

In [ ]:
#final_game_dict = transform_all_data(current_file)

In [ ]:
#final_game_dict['hands'][-2]['preflop_actions']

# Finalização

## Escrita JSON

In [ ]:
def write_json(file):
  final_game_dict = transform_all_data(file)

  platform = final_game_dict['match_metadata']['platform']
  match_type = final_game_dict['match_metadata']['match_type']
  match_id = final_game_dict['match_metadata']['id']

  json_data = json.dumps(final_game_dict)
  json_file_name = f"{HOJE}_{platform}_{match_type}_{match_id}"

  with open(f"{JSON_FOLDER_V2}{json_file_name}.json", "w") as file:
    json.dump(json_data, file)

  print(f"Arquivo salvo: {json_file_name}")

## Rodar em todos os arquivos

In [ ]:
for file in ms_sng_files:
  write_json(file)

Arquivo salvo: 2024-01-12_PokerStars_tournament_3491017877
Arquivo salvo: 2024-01-12_PokerStars_tournament_3492452250
Arquivo salvo: 2024-01-12_PokerStars_tournament_3494321597
Arquivo salvo: 2024-01-12_PokerStars_tournament_3495240506
Arquivo salvo: 2024-01-12_PokerStars_tournament_3495734268
Arquivo salvo: 2024-01-12_PokerStars_tournament_3496550016
Arquivo salvo: 2024-01-12_PokerStars_tournament_3496569097
Arquivo salvo: 2024-01-12_PokerStars_tournament_3496576671
Arquivo salvo: 2024-01-12_PokerStars_tournament_3496788900
Arquivo salvo: 2024-01-12_PokerStars_tournament_3496976846
Arquivo salvo: 2024-01-12_PokerStars_tournament_3497280061
Arquivo salvo: 2024-01-12_PokerStars_tournament_3497303233
Arquivo salvo: 2024-01-12_PokerStars_tournament_3497738639
Arquivo salvo: 2024-01-12_PokerStars_tournament_3498197391
Arquivo salvo: 2024-01-12_PokerStars_tournament_3498220037
Arquivo salvo: 2024-01-12_PokerStars_tournament_3498597122
Arquivo salvo: 2024-01-12_PokerStars_tournament_34986044

# Outros

## Testes

## Backup

In [ ]:
# Extrai o arquivo de texto e faz um tratamento básico
# Retorna uma lista de strings em que cada item é uma linha do texto

#source_file = "HH20221204 T3510653215 No Limit Hold'em $0.44 + $0.06.txt" # um dos menores torneios que eu joguei, com só 4 mãos. é o que mais tô usando pra testes
#source_file = "HH20221122 T3504217006 No Limit Hold'em $0.44 + $0.06.txt" # um dos MAIORES torneios que eu tenho. bom pra outros testes


def_clean_txt_file = '''

def clean_txt_file(file):
  complete_filepath = f"{SOURCE_FOLDER}{file}"
  with open(complete_filepath) as raw_text_game_data:
    table_strings = raw_text_game_data.readlines()
    clean_table_strings = [line.strip() for line in table_strings]
    list_no_empty_lines = list(filter(None, clean_table_strings))
    return list_no_empty_lines

'''

In [ ]:
# Separa o blocão de texto anterior em mãos, criando uma lista pra cada mão
# Retorna uma lista de listas com as linhas do texto

def_get_raw_hands = '''

def get_raw_hands():
  no_empty_lines = clean_txt_file(current_file)
  raw_hands_data = []

  table_hand_counter = 0
  hand_list = []
  for line in no_empty_lines:
    if "PokerStars" in line:
      if table_hand_counter != 0:
        raw_hands_data.append(hand_list)
        hand_list = []
      table_hand_counter += 1
      hand_list.clear() # redundante? n lembro e pregs de debugar
    hand_list.append(line)
  raw_hands_data.append(hand_list)
  return raw_hands_data

#get_raw_hands()

'''

In [ ]:
# Separa as mãos em estágios (setup, pre-flop etc)
# retorna uma lista de dicionários. key = stage, value = raw lines

def_break_hands_stages = '''

def break_hands_stages():
  raw_hands_data = get_raw_hands()
  raw_hands_stages = []

  for hand in raw_hands_data:
    raw_dict = {
      'Metadata': [],
      'Setup': [],
      'Pre_flop': [],
      'Flop': [],
      'Turn': [],
      'River': [],
      'Showdown': [],
      'Summary': []
    }
    for line in hand:
      if "PokerStars" in line:
        stage = 'Setup'
      if "*** HOLE CARDS ***" in line:
        stage = "Pre_flop"
      if "*** FLOP ***" in line:
        stage = "Flop"
      if "TURN" in line:
        stage = "Turn"
      if "RIVER" in line:
        stage = "River"
      if "SHOW DOWN" in line:
        stage = "Showdown"
      if "SUMMARY" in line:
        stage = "Summary"
      raw_dict[stage].append(line)
    raw_hands_stages.append(raw_dict)

  # tira as duas primeiras linhas do Setup e joga pro Metadata. Achei mais fácil
  for hand in raw_hands_stages:
    hand['Metadata'].append(hand['Setup'][0])
    hand['Metadata'].append(hand['Setup'][1])
    hand['Setup'].pop(0)
    hand['Setup'].pop(0)

  return raw_hands_stages

#pprint.pprint(break_hands_stages(), sort_dicts=False)

'''